# Build Chatbot
`11_chatbot.ipynb`
- https://python.langchain.com/docs/tutorials/chatbot/

In [ ]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

messages = [
    HumanMessage(content='Hi!, I am bob.'),
    AIMessage(content='Hello bob. how can I help you.'),
    HumanMessage(content='Say my name.')
]

llm = ChatOpenAI(model='gpt-4.1-nano', temperature=0)

res = llm.invoke(messages)

res.pretty_print()

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, END, MessagesState, StateGraph

# Graph Builider
builder = StateGraph(state_schema=MessagesState)

# Node
def simple_node(state: MessagesState):
    res = llm.invoke(state['messages'])
    return {'messages': res}

builder.add_node('simple_node', simple_node)

# Edge (Node 끼리 연결)
builder.add_edge(START, 'simple_node')
builder.add_edge('simple_node', END)

# Memory (대화내역 기록)
memory = MemorySaver()

# Graph (그래프 생성)
graph = builder.compile(checkpointer=memory)

In [ ]:
# 설정(conf, config, configuration -> 설정)
config = {'configurable': {'thread_id': 'abc123'}}  # 채팅방 아이디 (바뀌면 다른 대화가 된다.)

graph.invoke({'messages': messages}, config=config)

In [ ]:
import uuid

u_id = uuid.uuid1()
print(u_id)

config = {'configurable': {'thread_id': '가나다123'}}  # 채팅방 아이디 -> 추후에는 UUID 형식으로 생성
messages = [
    HumanMessage(content='say my name.')
]
graph.invoke({'messages': messages}, config=config)

## Langgraph + `PromptTemplate`

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages([
    ('system', '너는 유능한 어시스턴트야. 너의 능력을 최대한 활용해서 답을 해봐.'),
    MessagesPlaceholder(variable_name='messages')  # 모든 저장된 대화 내용(최신것 포함)
])

# 실행 예시
for msg in prompt_template.invoke({'messages': ['hi']}).messages:
    print(msg)

## State 확장

In [39]:
# 내장된 MessagesState를 확장해서 사용
class MyState(MessagesState):
    # 상속받아서 이미 key 'messages'는 있음
    # messages: Annotated[list[AnyMessage], add_messages]
    lang: str

builder = StateGraph(state_schema=MyState)

prompt_template = ChatPromptTemplate.from_messages([
    ('system', '너는 유능한 어시스턴트야. 너의 능력을 최대한 활용해서 답을 해봐. {lang} 언어로 답해.'),
    MessagesPlaceholder(variable_name='messages')  # 모든 저장된 대화 내용(최신것 포함)
])

def simple_node(state: MyState):
    # prompt 추가.
    chain = prompt_template | llm  # 체인 방식
    res = chain.invoke(state)

    return {'messages': res}

builder.add_node('simple_node', simple_node)

builder.add_edge(START, 'simple_node')
builder.add_edge('simple_node', END)

memory = MemorySaver()

graph = builder.compile(checkpointer=memory)

In [40]:
config = {'configurable': {'thread_id': 'abc1'}} 
state = {
    'messages': [HumanMessage(content='나는 서울에 있어')],
    'lang': 'Spanish'
}

res = graph.invoke(state, config)

for msg in res['messages']:
    msg.pretty_print()

================================ Human Message =================================

나는 서울에 있어
================================== Ai Message ==================================

¡Qué genial! ¿En qué puedo ayudarte mientras estás en Seúl?


## 대화 기록 관리하기
대확 내역을 관리하지 않으면, LLM의 컨텍스트 윈도우(입력 최대치)를 넘어가 버림.


In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, trim_messages

# trim - 정리하다
trimmer = trim_messages(
    strategy='last',      # 최신 메세지들을
    max_tokens=65,         # 최대 65토큰 까지만 허용,
    token_counter=llm,    # llm 모델에 맞춰서 토큰 세고
    include_system=True,  # system 프롬프트는 포함(정리X)
    allow_partial=False,  # 메세지 중간에서 자르지는 말고
    start_on='human',     # 잘린 메세지의 첫번째는 사람 메세지가 되도록
    
)

messages = [
    SystemMessage(content="you're a good assistant"),

    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)  # 시스템메시지 -> 2+2 부터 등장

[HumanMessage(content='messages', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='lang', additional_kwargs={}, response_metadata={})]

In [63]:
class MyState(MessagesState):
    # messages: ~~
    lang: str


builder = StateGraph(state_schema=MyState)

prompt_template = ChatPromptTemplate.from_messages([
    ('system', '너는 유능한 어시스턴트야. 너의 능력을 최대한 활용해서 답을 해봐. {lang} 언어로 답해.'),
    MessagesPlaceholder(variable_name='messages')  # 모든 저장된 대화 내용(최신것 포함)
])


def simple_node(state: MyState):
    # 메세지 정리 -> 프롬프트 생성 -> LLM 답변
    print('정리 전 메시지 개수: ', len(state['messages']))
    trimmed_messages = trimmer.invoke(state['messages'])
    print('정리 후 메시지 개수: ', len(trimmed_messages))
    print('********************************************************')
    for m in trimmed_messages:
        print('\t', m.pretty_print())
    print('********************************************************')
    
    # 체인 생성
    chain = prompt_template | llm
    
    # 정리된 메세지로 state 교체 후, 체인 실행
    state['messages'] = trimmed_messages
    res = chain.invoke(state)

    return {'messages': res}


builder.add_node('simple_node', simple_node)

builder.add_edge(START, 'simple_node')
builder.add_edge('simple_node', END)

memory = MemorySaver()

graph = builder.compile(checkpointer=memory)

In [ ]:
config = {'configurable': {'thread_id': 'abc1'}} 
state = {
    'messages': [HumanMessage(content='아까 추천해준 메뉴들 설명해줘')],
    'lang': '한국어'
}

res = graph.invoke(state, config)

# 채팅 저장은 잘 되지만, 대화내용이 잘려서 들어가는걸 확인 가능!
for msg in res['messages']:
    msg.pretty_print()

정리 전 메시지 개수:  3
정리 후 메시지 개수:  1
********************************************************
================================ Human Message =================================

아까 추천해준 메뉴들 설명해줘
	 None
********************************************************
================================ Human Message =================================

배고파 메뉴 추천해줘
================================== Ai Message ==================================

배고프시다니! 어떤 종류의 음식을 좋아하시는지 알려주시면 더 딱 맞는 추천을 드릴 수 있는데요. 그래도 다양한 메뉴를 추천해드릴게요.

1. 한식:
   - 김치찌개와 밥
   - 불고기와 상추쌈
   - 비빔밥

2. 일식:
   - 초밥 세트
   - 라멘
   - 돈부리(덮밥)

3. 중식:
   - 짜장면과 탕수육
   - 마파두부와 공기밥
   - 깐풍기

4. 간단한 간식/샌드위치:
   - 클럽 샌드위치
   - 햄치즈 샌드위치
   - 김밥

5. 패스트푸드:
   - 치킨버거와 감자튀김
   - 피자 한 판

혹시 특정한 음식 종류나 선호하는 맛이 있으시면 알려주세요! 더 맞춤형 추천도 가능해요.
================================ Human Message =================================

아까 추천해준 메뉴들 설명해줘
================================== Ai Message ==================================

물론입니다! 어떤 메뉴들을 추천받았는지 구체적으로 알려주시면, 각각의 메뉴에 대